# 1. Query tools

This Section introduces three methods to query the ALMA archive:

- [1.1 - Query by target name (*alminer.target*)](#1.1-Query-by-target-name)<br>
- [1.2 - Query a catalog (*alminer.catalog*)](#1.2-Query-by-position)<br>
- [1.3 - Query by ALMA keywords (*alminer.keysearch*)](#1.3-Query-by-ALMA-keywords)

<h3>General notes about the querying functions:</h3>

* All querying functions search the ALMA archive for public data by default. To include both public and proprietary data in the search, set *public=None*. Similarly, to search for only propietary data, set *public=False*. 
* All querying functions search the ALMA archive for both published and unpublished data. To include only unpublished data, set *published=False*.
* The querying functions will by default print a summary of the observations, including a list of target names. For large queries, it is useful to turn this feature off in order to not have a long list of targets printed to screen. To turn off this feature, simply set *print_targets=False*.
* The queries return all possible observations in [PANDAS DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html) format that can be used to further narrow down your search as demonstrated in [Section 2](2_filter_explore.ipynb).

<h3>Load libraries </h3>

In [34]:
import alminer
import pandas
from astropy.io import ascii

## 1.1 Query by target name

The `alminer.target` function allows one to query objects by name. This function uses the Astropy [SESAME resolver](http://cds.u-strasbg.fr/cgi-bin/Sesame) which searches multiple databases (Simbad, NED, VizieR) to obtain the coordinates of the object of interest, and then queries the ALMA archive for those coordinates and a search radius around them. The search radius is by default 1.0 arcminute, but can be modified using the *search_radius* keyword (in arcmin units). 

<style>
summary > * {
  display: inline;
</style>

<br>
<details style="background-color:#f8f8f8;">
<summary style="display:list-item;background-color:#f8f8f8;">Details of <code>target</code> function</summary>
    
* <u>Description</u>: Query targets by name. <br>
    * This is done by using the astropy SESAME resolver to get the target's coordinates and then the ALMA archive is queried for those coordinates and a search_radius around them. The SESAME resolver searches multiple databases (Simbad, NED, VizieR) to parse names commonly found throughout literature and returns their coordinates. If the target is not resolved in any of these databases, consider using the `keysearch` function and query the archive using the 'target_name' keyword (e.g. `keysearch({'target_name': sources})`).<br>
<br>
* <u>Command</u>:<br>
    * alminer.**target**(_sources, search_radius=1., point=True, public=True, published=None, print_query=False, print_targets=True_)<br>
<br>
* <u>Parameters</u>:<br>
    * **sources (list of str)** : list of sources by name. (IMPORTANT: sources names must be identified by at least one of Simbad, NED, or Vizier) <br> 
    * **search_radius (float, optional, default: 1 arcmin)** : Search radius (in arcmin) around the source coordinates. <br>
    * **point (bool, optional, default: True)** : Search whether the phase center of the observations is contained within the search_radius (*point=True*) or whether any part of the observed region overlaps with the cone extending the search_radius (*point=False*). Note that *point=True* is much faster than *point=False* but the latter should be used if one is interested in searching for mosaics.<br>
    * **public (bool, optional, default: True)** : Search for public data (*public=True*), proprietary data (*public=False*), or both public and proprietary data (*public=None*). <br>
    * **published (bool, optional, default: None)** : Search for published data only (*published=True*), unpublished data only (*published=False*), or both published and unpublished data (*published=None*). <br>
    * **print_query (bool, optional, default: False)** : Print the ADQL TAP query to the terminal. <br>
    * **print_targets (bool, optional, default: True)** : Print a list of targets with ALMA data (ALMA source names) to the terminal. <br>
 <br>
* <u>Returns</u>:<br>
    *  **pandas.DataFrame** containing the query results.

</details>

### Example 1.1.1: query two sources by name

In [22]:
myquery = alminer.target(['Orion KL', "AB Aur"])

alminer.target results 
Target = Orion KL
--------------------------------
Number of projects = 27
Number of observations = 63
Number of unique subbands = 226
Total number of subbands = 272
23 target(s) with ALMA data = ['Orion H2O maser outburst', 'OrionKL', 'Orion_Source_I', 'Orion KL', 'Orion-KL', 'Orion_KL', 'BN-KL', 'OrionKL-SV', 'f15', 'f14', 'OMC1_NW', 'Orion1', 'OMC1_SE', 'BN', 'f23', 'Orion', 'Orion_BNKL_source_I', 'OMC-1_Region2', 'orion_kl', 'orion-IRc2', 'Orion_KL_Field_1_Orion_Hot_Core', 'Orion_KL_Field_3_North-west_Clump', 'Orion_KL_Field_2_SMA1']
--------------------------------
Target = AB Aur
--------------------------------
Number of projects = 3
Number of observations = 3
Number of unique subbands = 17
Total number of subbands = 17
3 target(s) with ALMA data = ['ab_aurigae', 'AB_Aur', 'AB_Auriga']
--------------------------------


Now let's increase the search radius:

In [23]:
myquery = alminer.target(['Orion KL', "AB Aur"], search_radius=2.0)

alminer.target results 
Target = Orion KL
--------------------------------
Number of projects = 35
Number of observations = 100
Number of unique subbands = 283
Total number of subbands = 451
46 target(s) with ALMA data = ['HC602_HC606_HC608', 'Orion H2O maser outburst', 'OrionField1-2', 'OrionKL', 'Orion_Source_I', 'HC672', 'OrionField1-1', 'OrionField2', 'Orion KL', 'OMC-1S', 'Orion-KL', '104', 'Orion_KL', '71', 'BN-KL', 'OrionKL-SV', 'f11', 'f12', 'f15', 'f13', 'f14', 'OMC1_SE', 'OMC1_NW', 'f10', 'Orion1', 'f8', 'f4', 'f5', 'f1', 'f7', 'f3', 'BN', 'f23', 'Orion', 'Orion_BNKL_source_I', 'f16', 'OMC-1_Region4', 'OMC-1_Region2', 'orion_kl', 'ONC_Mosaic', 'orion-IRc2', 'Orion_KL_Field_1_Orion_Hot_Core', 'ONC', 'Orion_KL_Field_3_North-west_Clump', 'Orion_KL_Field_2_SMA1', 'ORS-8']
--------------------------------
Target = AB Aur
--------------------------------
Number of projects = 3
Number of observations = 3
Number of unique subbands = 17
Total number of subbands = 17
3 target(s) with A

### Example 1.1.2: query a list of objects by name

First create a catalog or a list of object names. In this example, the catalog `Sample_cat.dat` has the following content:
```
 Name     RA      DEC   
------ -------- --------
AB_Aur  73.9412  30.5511
AK_Sco 253.6867 -36.8886
AS_310 278.3383  -4.9683
AS_470 324.0592  57.3586
AS_477 328.1421  47.2289
```

Note that the column that is used is the *Name* column and the coordinates are ignored in this example.

In [24]:
mylist = ascii.read("Sample_cat.dat", header_start=0, data_start=1)
myquery = alminer.target(mylist['Name'])

alminer.target results 
Target = AB_Aur
--------------------------------
Number of projects = 3
Number of observations = 3
Number of unique subbands = 17
Total number of subbands = 17
3 target(s) with ALMA data = ['ab_aurigae', 'AB_Aur', 'AB_Auriga']
--------------------------------
Target = AK_Sco
--------------------------------
Number of projects = 3
Number of observations = 3
Number of unique subbands = 12
Total number of subbands = 12
2 target(s) with ALMA data = ['AK_Sco', 'HIP_82747']
--------------------------------
Target = AS_310
--------------------------------
No observations found.
--------------------------------
Target = AS_470
--------------------------------
No observations found.
--------------------------------
Target = AS_477
--------------------------------
No observations found.
--------------------------------


### Example 1.1.3: include proprietary data

In [25]:
myquery = alminer.target(mylist['Name'], public=None)

alminer.target results 
Target = AB_Aur
--------------------------------
Number of projects = 4
Number of observations = 5
Number of unique subbands = 25
Total number of subbands = 25
3 target(s) with ALMA data = ['ab_aurigae', 'AB_Aur', 'AB_Auriga']
--------------------------------
Target = AK_Sco
--------------------------------
Number of projects = 4
Number of observations = 4
Number of unique subbands = 16
Total number of subbands = 16
2 target(s) with ALMA data = ['AK_Sco', 'HIP_82747']
--------------------------------
Target = AS_310
--------------------------------
No observations found.
--------------------------------
Target = AS_470
--------------------------------
No observations found.
--------------------------------
Target = AS_477
--------------------------------
No observations found.
--------------------------------


### Example 1.1.4: account for mosaics

The `alminer.target` function will by default search whether the phase center of any ALMA observations lie within the search radius around the target of interest's position. To search whether any part of the observed region overlaps with the cone extending the search radius, one can set *point=False*. Note that this option is slower, hence it's not the dafault mode.

In [26]:
myquery = alminer.target(['Orion KL'], point=False)

alminer.target results 
Target = Orion KL
--------------------------------
Number of projects = 35
Number of observations = 90
Number of unique subbands = 283
Total number of subbands = 399
40 target(s) with ALMA data = ['HC602_HC606_HC608', 'Orion H2O maser outburst', 'OrionField1-2', 'OrionKL', 'Orion_Source_I', 'OrionField1-1', 'Orion KL', 'Orion-KL', '104', 'Orion_KL', 'BN-KL', 'OrionKL-SV', 'f11', 'f12', 'f15', 'f13', 'f14', 'OMC1_SE', 'OMC1_NW', 'f10', 'Orion1', 'f1', 'f3', 'BN', 'f23', 'Orion', 'OMC-1_Region5', 'Orion_BNKL_source_I', 'f16', 'OMC-1_Region4', 'OMC-1_Region2', 'orion_kl', 'OMC-1', 'ONC_Mosaic', 'orion-IRc2', 'Orion_KL_Field_1_Orion_Hot_Core', 'ONC', 'Orion_KL_Field_3_North-west_Clump', 'Orion_KL_Field_2_SMA1', 'ORS-8']
--------------------------------


## 1.2 Query by position

The `alminer.conesearch` and `alminer.catalog` functions can be used to directly query the ALMA archive by positions in the sky and a search radius around them. The right ascension and declinations must be given in units of degrees (ICRS). You can use the [Astropy coordinates package](https://docs.astropy.org/en/stable/coordinates/index.html) to convert your desired coordinates to degrees.

<style>
summary > * {
  display: inline;
</style>

<br>
<details style="background-color:#f8f8f8;">
<summary style="display:list-item;background-color:#f8f8f8;">Details of <code>conesearch</code> function</summary>

* <u>Description</u>: Query the ALMA archive for a given position and radius around it.<br>
    <br>
* <u>Command</u>:<br>
    * alminer.**conesearch**(_ra, dec, search_radius=1., point=True, public=True, published=None, print_targets=True, print_query=False_)<br>
<br>
* <u>Parameters</u>:<br>
    * **ra (float)** : Right ascension in degrees (ICRS). <br>
    * **dec (float)** : Declination in degrees (ICRS). <br>
    * **search_radius (float, optional, default: 1 arcmin)** : Search radius (in arcmin) around the source coordinates. <br>
    * **point (bool, optional, default: True)** : Search whether the phase center of the observations is contained within the search_radius (*point=True*) or whether any part of the observed region overlaps with the cone extending the search_radius (*point=False*). Note that *point=True* is much faster than *point=False* but the latter should be used if one is interested in searching for mosaics.<br>
    * **public (bool, optional, default: True)** : Search for public data (*public=True*), proprietary data (*public=False*), or both public and proprietary data (*public=None*). <br>
    * **published (bool, optional, default: None)** : Search for published data only (*published=True*), unpublished data only (*published=False*), or both published and unpublished data (*published=None*). <br>
    * **print_query (bool, optional, default: False)** : Print the ADQL TAP query to the terminal. <br>
    * **print_targets (bool, optional, default: True)** : Print a list of targets with ALMA data (ALMA source names) to the terminal. <br>
 <br>
* <u>Returns</u>:<br>
    *  **pandas.DataFrame** containing the query results.<br>
</details>

<details style="background-color:#f8f8f8;">
<summary style="display:list-item;background-color:#f8f8f8;">Details of <code>catalog</code> function</summary>

* <u>Description</u>: Query the ALMA archive for a list of coordinates or a catalog of sources based on their coordinates. <br>
<br>
* <u>Command</u>:<br>
    * alminer.**catalog**(_target_df, search_radius=1., point=True, public=True, published=None, print_query=False, print_targets=True_)<br>
<br>
* <u>Parameters</u>:<br>
    * **target_df (pandas.DataFrame)** : DataFrame with source names and coordinates; it must have at least the following three columns: "*Name*", "*RAJ2000*", "*DEJ2000*". "*Name*" describes the target name (can be numbers or dummy names), "*RAJ2000*" is the right ascension in degrees (ICRS), "*DEJ2000*" is the declination in degrees (ICRS) <br>
    * **search_radius (float, optional, default: 1 arcmin)** : Search radius (in arcmin) around the source coordinates. <br>
    * **point (bool, optional, default: True)** : Search whether the phase center of the observations is contained within the search_radius (*point=True*) or whether any part of the observed region overlaps with the cone extending the search_radius (*point=False*). Note that *point=True* is much faster than *point=False* but the latter should be used if one is interested in searching for mosaics.<br>
    * **public (bool, optional, default: True)** : Search for public data (*public=True*), proprietary data (*public=False*), or both public and proprietary data (*public=None*). <br>
    * **published (bool, optional, default: None)** : Search for published data only (*published=True*), unpublished data only (*published=False*), or both published and unpublished data (*published=None*). <br>
    * **print_query (bool, optional, default: False)** : Print the ADQL TAP query to the terminal. <br>
    * **print_targets (bool, optional, default: True)** : Print a list of targets with ALMA data (ALMA source names) to the terminal. <br>
 <br>
* <u>Returns</u>:<br>
    *  **pandas.DataFrame** containing the query results.
</details>

### Example 1.2.1:  query an object by its coordinates (RA, Dec)

In [27]:
myquery = alminer.conesearch(ra=201.365063, dec=-43.019112, search_radius=10.0)

--------------------------------
Number of projects = 23
Number of observations = 74
Number of unique subbands = 190
Total number of subbands = 301
9 target(s) with ALMA data = ['CenA', 'J1325-430', 'Centaurus_a', 'J1325-4301', 'Centaurus A', 'Centaurus_A', '3FGL_J1325.4-4301', 'NGC_5128', 'Cen_A']
--------------------------------


### Example 1.2.2:  query a catalog of objects by their coordinates (RA, Dec)

Let's first import a catalog, for example the catalog of Spitzer YSOs in Orion from Megeath et al. (2009), and create a PANDAS DataFrame using rows 866 to 869 of this catalog. Then use the `alminer.catalog` function to query the ALMA archive for each target in the DataFrame.

In [28]:
Spitzer = ascii.read("Spitzer_sample.dat", header_start=0, data_start=866, data_end=869)

mycat =  {"Name": Spitzer['Seq'], 
          "RAJ2000" : Spitzer["RA2000"], 
          "DEJ2000" : Spitzer["DEC2000"]}

mycat = pandas.DataFrame(mycat)

myquery = alminer.catalog(mycat, search_radius=1.)

alminer.catalog results
Target = 866
--------------------------------
Number of projects = 1
Number of observations = 1
Number of unique subbands = 4
Total number of subbands = 4
1 target(s) with ALMA data = ['M12_866']
--------------------------------
Target = 867
--------------------------------
Number of projects = 1
Number of observations = 1
Number of unique subbands = 4
Total number of subbands = 4
1 target(s) with ALMA data = ['M12_867']
--------------------------------
Target = 868
--------------------------------
Number of projects = 2
Number of observations = 2
Number of unique subbands = 8
Total number of subbands = 8
1 target(s) with ALMA data = ['HOPS-172']
--------------------------------


## 1.3 Query by ALMA keywords

Query the ALMA archive for any (string-type) keywords defined in ALMA TAP system (see table below or [here](../pages/query_keywords.ipynb)). 

The power of this function is in combining keywords. When multiple keywords are provided, they are queried using 'AND' logic, while when multiple values are provided for a given keyword, they are queried using 'OR' logic. For example,

 * `alminer.keysearch({"proposal_abstract": ["high-mass star formation outflow disk"]})`
will query the archive for projects with the words "high-mass" AND "star" AND "formation" AND "outflow" AND "disk" in their proposal abstracts.
<br>

 * `alminer.keysearch({"proposal_abstract": ["high-mass", "star", "formation", "outflow", "disk"]})`
will query the archive for projects with the words "high-mass" OR "star" OR "formation" OR "outflow" OR "disk" in their proposal abstracts.
<br>

 * `alminer.keysearch({"proposal_abstract": ["star formation"], "scientific_category":['Galaxy evolution']})`
will query the archive for projects with the words "star" AND "formation" in their proposal abstracts AND projects that are within the scientific_category of 'Galaxy evolution'.

<style>
summary > * {
  display: inline;
</style>

<details style="background-color:#f8f8f8;">
<summary style="display:list-item;background-color:#f8f8f8;">Details of <code>keysearch</code> function</summary>

* <u>Description</u>: Query the ALMA archive for any (string-type) keywords defined in ALMA TAP system. <br>
<br>
* <u>Command</u>:<br>
    * alminer.**keysearch**(_search_dict, public=True, published=None, print_query=False, print_targets=True_)<br>
<br>
* <u>Parameters</u>:<br>
    * **search_dict (dict[str, list of str])** : Dictionary of keywords in the ALMA archive and their values. Values must be formatted as a list of strings. For a list of valid keywords see [table](#TAP_table) below. <br> 
    * **public (bool, optional, default: True)** : Search for public data (*public=True*), proprietary data (*public=False*), or both public and proprietary data (*public=None*). <br>
    * **published (bool, optional, default: None)** : Search for published data only (*published=True*), unpublished data only (*published=False*), or both published and unpublished data (*published=None*). <br>
    * **print_query (bool, optional, default: False)** : Print the ADQL TAP query to the terminal. <br>
    * **print_targets (bool, optional, default: True)** : Print a list of targets with ALMA data (ALMA source names) to the terminal. <br>
 <br>
* <u>Returns</u>:<br>
    *  **pandas.DataFrame** containing the query results.
</details>

<style>
summary > * {
  display: inline
    }
</style>

<details>
<summary style="display:list-item">Table of possible query keywords  <a name="TAP_table" ></a></summary>

<table  align="left" class="docutils align-default">
    <thead>
    <tr style="border-bottom:1px solid black"><th style="text-align:left" >ALMA query keyword</th><th style="text-align:left" >Type</th><th style="text-align:left" >Description</th></tr>
    </thead>
    <tbody>
    <tr><td style="text-align:left" >access_format</td><td  style="text-align:left">char(9)</td><td  style="text-align:left">Content format of the data</td></tr>
    <tr><td style="text-align:left" >access_url</td><td  style="text-align:left">char(72*)</td><td  style="text-align:left">URL to download the data</td></tr>
    <tr><td style="text-align:left" >antenna_arrays</td><td  style="text-align:left">char(660*)</td><td  style="text-align:left">Blank-separated list of Pad:Antenna pairs, i.e., A109:DV09 J504:DV02 J505:DV05 <br> for antennas DV09, DV02 and DV05 sitting on pads A109, J504, and J505, respectively.</td></tr>
    <tr><td style="text-align:left" >asdm_uid</td><td  style="text-align:left">char(32*)</td><td  style="text-align:left">UID of the ASDM containing this Field.</td></tr>
    <tr><td style="text-align:left" >authors</td><td  style="text-align:left">char(4000*)</td><td  style="text-align:left">Full list of first author and all co-authors</td></tr>
    <tr><td style="text-align:left" >band_list</td><td  style="text-align:left">char(30*)</td><td  style="text-align:left">Space delimited list of bands</td></tr>
    <tr><td style="text-align:left" >bib_reference</td><td  style="text-align:left">char(30*)</td><td  style="text-align:left">Bibliography code</td></tr>
    <tr><td style="text-align:left" >data_rights</td><td  style="text-align:left">char(11)</td><td  style="text-align:left">Access to data.</td></tr>
    <tr><td style="text-align:left" >dataproduct_type</td><td  style="text-align:left">char(5*)</td><td  style="text-align:left">type of product</td></tr>
    <tr><td style="text-align:left" >facility_name</td><td  style="text-align:left">char(3)</td><td  style="text-align:left">telescope name</td></tr>
    <tr><td style="text-align:left" >first_author</td><td  style="text-align:left">char(256*)</td><td  style="text-align:left">The first author as provided by <a href="http://telbib.eso.org">telbib.eso.org</a>.</td></tr>
    <tr><td style="text-align:left" >frequency_support</td><td  style="text-align:left">char(4000*)</td><td  style="text-align:left">All frequency ranges used by the field</td></tr>
    <tr><td style="text-align:left" >group_ous_uid</td><td  style="text-align:left">char(64*)</td><td  style="text-align:left">Group OUS ID</td></tr>
    <tr><td style="text-align:left" >instrument_name</td><td  style="text-align:left">char(4)</td><td  style="text-align:left">instrument name</td></tr>
    <tr><td style="text-align:left" >is_mosaic</td><td  style="text-align:left">char(1)</td><td  style="text-align:left">Flag to indicate if this ASDM represents a mosaic or not.</td></tr>
    <tr><td style="text-align:left" >lastModified</td><td  style="text-align:left">char(*)</td><td  style="text-align:left">Time stamp of last modification of the metadata</td></tr>
    <tr><td style="text-align:left" >member_ous_uid</td><td  style="text-align:left">char(64*)</td><td  style="text-align:left">Member OUS ID</td></tr>
    <tr><td style="text-align:left" >o_ucd</td><td  style="text-align:left">char(35)</td><td  style="text-align:left">UCD describing the observable axis (pixel values)</td></tr>
    <tr><td style="text-align:left" >obs_collection</td><td  style="text-align:left">char(4)</td><td  style="text-align:left">short name for the data collection</td></tr>
    <tr><td style="text-align:left" >obs_creator_name</td><td  style="text-align:left">char(256*)</td><td  style="text-align:left">case-insensitive partial match over the full PI name. Wildcards can be used</td></tr>
    <tr><td style="text-align:left" >obs_id</td><td  style="text-align:left">char(64*)</td><td  style="text-align:left">internal dataset identifier</td></tr>
    <tr><td style="text-align:left" >obs_publisher_did</td><td  style="text-align:left">char(33*)</td><td  style="text-align:left">publisher dataset identifier</td></tr>
    <tr><td style="text-align:left" >obs_release_date</td><td  style="text-align:left">char(*)</td><td  style="text-align:left">timestamp of date the data becomes publicly available</td></tr>
    <tr><td style="text-align:left" >obs_title</td><td  style="text-align:left">char(256*)</td><td  style="text-align:left">Case-insensitive search over the project title</td></tr>
    <tr><td style="text-align:left" >pol_states</td><td  style="text-align:left">char(64*)</td><td  style="text-align:left">polarization states present in the data</td></tr>
    <tr><td style="text-align:left" >proposal_abstract</td><td  style="text-align:left">char(4000*)</td><td  style="text-align:left">Text search on the proposal abstract. Only abstracts will be returned <br> which contain the given text. The search is case-insensitive.</td></tr>
    <tr><td style="text-align:left" >proposal_authors</td><td  style="text-align:left">char(2000*)</td><td  style="text-align:left">Full name of CoIs.</td></tr>
    <tr><td style="text-align:left" >proposal_id</td><td  style="text-align:left">char(64*)</td><td  style="text-align:left">Identifier of proposal to which NO observation belongs.</td></tr>
    <tr><td style="text-align:left" >pub_abstract</td><td  style="text-align:left">char(4000*)</td><td  style="text-align:left">Case insensitive text search through the abstract of the publication.</td></tr>
    <tr><td style="text-align:left" >pub_title</td><td  style="text-align:left">char(256*)</td><td  style="text-align:left">Case insensitive search through the title of the publication.</td></tr>
    <tr><td style="text-align:left" >qa2_passed</td><td  style="text-align:left">char(1)</td><td  style="text-align:left">Quality Assessment 2 status: does the Member / Group OUS  <br> fulfil the PI's requirements?</td></tr>
    <tr><td style="text-align:left" >s_region</td><td  style="text-align:left">char(*)</td><td  style="text-align:left">region bounded by observation</td></tr>
    <tr><td style="text-align:left" >scan_intent</td><td  style="text-align:left">char(256*)</td><td  style="text-align:left">Scan intent list for the observed field.</td></tr>
    <tr><td style="text-align:left" >schedblock_name</td><td  style="text-align:left">char(128*)</td><td  style="text-align:left">Name of the Scheduling Block used as a template for executing the ASDM containing this Field.</td></tr>
    <tr><td style="text-align:left" >science_keyword</td><td  style="text-align:left">char(200*)</td><td  style="text-align:left">Chosen by the PI in the observing tool at the time of proposal submission. For an overview, <br> see <a href="https://almascience.nrao.edu/proposing/proposers-guide#section-63">Appendix D of the ALMA Proposer's Guide</a>. For a precise list, see <a href="#sci_keywords">Table of science keywords</a> below.</td></tr>
    <tr><td style="text-align:left" >science_observation</td><td  style="text-align:left">char(1)</td><td  style="text-align:left">Flag to indicate whether this is a science observation.</td></tr>
    <tr><td style="text-align:left" >scientific_category</td><td  style="text-align:left">char(200*)</td><td  style="text-align:left">Chosen by the PI in the observing tool at the time of proposal submission. For an overview, <br>  see <a href="https://almascience.nrao.edu/proposing/proposers-guide#section-63">Appendix D of the ALMA Proposer's Guide</a>. For a precise list, see <a href="#sci_category">Table of science categories</a> below.</td></tr>
    <tr><td style="text-align:left" >target_name</td><td  style="text-align:left">char(256*)</td><td  style="text-align:left">name of intended target</td></tr>
    <tr><td style="text-align:left" >type</td><td  style="text-align:left">char(16*)</td><td  style="text-align:left">Type flags.</td></tr>
    </tbody>
</table><br>
</details>

<style>
summary > * {
  display: inline;
</style>

<details>
<summary style="display:list-item">Table of possible ALMA science categories <a name="sci_category" ></a></summary>

<table  align="left" class="docutils align-default">   
<tr  style="border-bottom:1px solid black"><th style="text-align:left" >ALMA science category</th></tr>
<tr><td style="text-align:left" >Active galaxies</td></tr>
<tr><td style="text-align:left" >Cosmology</td></tr>
<tr><td style="text-align:left" >Disks and planet formation</td></tr>
<tr><td style="text-align:left" >Galaxy evolution</td></tr>
<tr><td style="text-align:left" >ISM and star formation</td></tr>
<tr><td style="text-align:left" >Local Universe</td></tr>
<tr><td style="text-align:left" >Solar system</td></tr>
<tr><td style="text-align:left" >Stars and stellar evolution</td></tr>
<tr><td style="text-align:left" >Sun</td></tr>
</table><br>
</details>

<style>
summary > * {
  display: inline;
</style>

<details>
<summary style="display:list-item">Table of possible ALMA science keywords <a name="sci_keywords" ></a></summary>

<table  align="left" class="docutils align-default">   
<tr  style="border-bottom:1px solid black" ><th style="text-align:left" >ALMA science keyword</th></tr>
<tr><td style="text-align:left" >Active Galactic Nuclei (AGN)/Quasars (QSO)</td></tr>
<tr><td style="text-align:left" >Astrochemistry</td></tr>
<tr><td style="text-align:left" >Asymptotic Giant Branch (AGB) stars</td></tr>
<tr><td style="text-align:left" >Black holes</td></tr>
<tr><td style="text-align:left" >Brown dwarfs</td></tr>
<tr><td style="text-align:left" >Cataclysmic stars</td></tr>
<tr><td style="text-align:left" >Cosmic Microwave Background (CMB)/Sunyaev-Zel'dovich Effect (SZE)"</td></tr>
<tr><td style="text-align:left" >Damped Lyman Alpha (DLA) systems</td></tr>
<tr><td style="text-align:left" >Debris disks</td></tr>
<tr><td style="text-align:left" >Disks around high-mass stars</td></tr>
<tr><td style="text-align:left" >Disks around low-mass stars</td></tr>
<tr><td style="text-align:left" >Dwarf/metal-poor galaxies</td></tr>
<tr><td style="text-align:left" >Early-type galaxies</td></tr>
<tr><td  style="text-align:left" >Evolved stars - Chemistry</td></tr>
<tr><td  style="text-align:left" >Evolved stars - Shaping/physical structure</td></tr>
<tr><td  style="text-align:left" >Evolved stars: Shaping/physical structure</td></tr>
<tr><td  style="text-align:left" >Exo-planets</td></tr>
<tr><td  style="text-align:left" >Galactic centres/nuclei</td></tr>
<tr><td  style="text-align:left" >Galaxy Clusters</td></tr>
<tr><td  style="text-align:left" >Galaxy chemistry</td></tr>
<tr><td  style="text-align:left" >Galaxy groups and clusters</td></tr>
<tr><td  style="text-align:left" >Galaxy structure & evolution</td></tr>
<tr><td  style="text-align:left" >Gamma Ray Bursts (GRB)</td></tr>
<tr><td  style="text-align:left" >Giant Molecular Clouds (GMC) properties</td></tr>
<tr><td  style="text-align:left" >Gravitational lenses</td></tr>
<tr><td  style="text-align:left" >HII regions</td></tr>
<tr><td  style="text-align:left" >High-mass star formation</td></tr>
<tr><td  style="text-align:left" >High-z Active Galactic Nuclei (AGN)</td></tr>
<tr><td  style="text-align:left" >Hypergiants</td></tr>
<tr><td  style="text-align:left" >Infra-Red Dark Clouds (IRDC)</td></tr>
<tr><td  style="text-align:left" >Inter-Stellar Medium (ISM)/Molecular clouds</td></tr>
<tr><td  style="text-align:left" >Intermediate-mass star formation</td></tr>
<tr><td  style="text-align:left" >Low-mass star formation</td></tr>
<tr><td  style="text-align:left" >Luminous Blue Variables (LBV)</td></tr>
<tr><td  style="text-align:left" >Luminous and Ultra-Luminous Infra-Red Galaxies (LIRG & ULIRG)</td></tr>
<tr><td  style="text-align:left" >Lyman Alpha Emitters/Blobs (LAE/LAB)</td></tr>
<tr><td  style="text-align:left" >Lyman Break Galaxies (LBG)</td></tr>
<tr><td  style="text-align:left" >Magellanic Clouds</td></tr>
<tr><td  style="text-align:left" >Main sequence stars</td></tr>
<tr><td  style="text-align:left" >Merging and interacting galaxies</td></tr>
<tr><td  style="text-align:left" >Outflows, jets, feedback</td></tr>
<tr><td  style="text-align:left" >Outflows, jets and ionized winds</td></tr>
<tr><td  style="text-align:left" >Photon-Dominated Regions (PDR)/X-Ray Dominated Regions (XDR)</td></tr>
<tr><td  style="text-align:left" >Post-AGB stars</td></tr>
<tr><td  style="text-align:left" >Pre-stellar cores</td></tr>
<tr><td  style="text-align:left" >Pulsars and neutron stars</td></tr>
<tr><td  style="text-align:left" >Solar system - Asteroids</td></tr>
<tr><td  style="text-align:left" >Solar system - Comets</td></tr>
<tr><td  style="text-align:left" >Solar system - Planetary atmospheres</td></tr>
<tr><td  style="text-align:left" >Solar system - Planetary surfaces</td></tr>
<tr><td  style="text-align:left" >Solar system - Trans-Neptunian Objects (TNOs)</td></tr>
<tr><td  style="text-align:left" >Spiral galaxies</td></tr>
<tr><td  style="text-align:left" >Starburst galaxies</td></tr>
<tr><td  style="text-align:left" >Starbursts, star formation</td></tr>
<tr><td  style="text-align:left" >Sub-mm Galaxies (SMG)</td></tr>
<tr><td  style="text-align:left" >Supernovae (SN) ejecta</td></tr>
<tr><td  style="text-align:left" >Surveys of galaxies</td></tr>
<tr><td  style="text-align:left" >The Sun</td></tr>
<tr><td  style="text-align:left" >Transients</td></tr>
<tr><td  style="text-align:left" >White dwarfs</td></tr>
</table>
</details>

### Example 1.3.1: query a list of ALMA target names that may not be in SIMBAD/NED/VizieR

In [29]:
myquery = alminer.keysearch({'target_name': ['GRB021004','SPT0319-47', 'G345']})

alminer.keysearch results 
--------------------------------
Number of projects = 16
Number of observations = 28
Number of unique subbands = 121
Total number of subbands = 179
10 target(s) with ALMA data = ['GRB021004', 'G345.5', 'SPT0319-47', 'G345.5043+00.3480', 'G345.49+1.47', 'G345.50+0.35', 'G345.6487+0.0089', 'G345.01', 'G345.11', 'G345.0029-0.2241']
--------------------------------


### Example 1.3.2: query a list of ALMA projects by their proposal IDs

In [30]:
myquery = alminer.keysearch({'proposal_id': ['2015.1.00664.S', '2016.1.00204.S']})

alminer.keysearch results 
--------------------------------
Number of projects = 2
Number of observations = 16
Number of unique subbands = 16
Total number of subbands = 64
16 target(s) with ALMA data = ['KMOS3DGS4-27882', 'KMOS3DCOS4-24763', 'KMOS3DU4-34138', 'KMOS3DCOS4-19680', 'KMOS3DCOS4-13701', 'KMOS3DCOS4-10347', 'KMOS3DCOS4-13174', 'KMOS3DCOS4-15820', 'KMOS3DCOS4-15813', 'KMOS3DU4-22227', 'KMOS3DU4-20547', 'KMOS3DU4-32147', 'KMOS3DGS4-24110', 'KMOS3DGS4-11016', 'KMOS3DGS4-25151', 'AK_Sco']
--------------------------------


### Example 1.3.3: query by words in the proposal abstract

Query the ALMA archive for proposals that have the words 'high-mass' AND 'star' AND 'formation' AND 'outflow', and do not print the long list of target names in the summary.

In [31]:
myquery = alminer.keysearch({'proposal_abstract': ['high-mass star formation outflow']}, 
                            print_targets=False)

alminer.keysearch results 
--------------------------------
Number of projects = 34
Number of observations = 419
Number of unique subbands = 416
Total number of subbands = 2799
Total number of targets with ALMA data = 239
--------------------------------


### Example 1.3.4: query by combination of keywords

Query the ALMA archive for proposals that have the words ('high-mass' AND 'star' AND 'formation' AND 'outflow') OR ('massive' AND 'star' AND 'formation') corresponding to the scientific category of 'Galaxy evolution'.

In [32]:
myquery = alminer.keysearch({'proposal_abstract': ['high-mass star formation', 'massive star formation'], 
                             'scientific_category':['Galaxy evolution']}, print_targets=False)

alminer.keysearch results 
--------------------------------
Number of projects = 119
Number of observations = 2056
Number of unique subbands = 2481
Total number of subbands = 8247
Total number of targets with ALMA data = 1376
--------------------------------


### Example 1.3.5: query for full polarization data

In [33]:
myquery = alminer.keysearch({'science_keyword':['disks around low-mass stars'], 
                             'pol_states':['XY', 'YX']}, print_targets=False)

alminer.keysearch results 
--------------------------------
Number of projects = 27
Number of observations = 48
Number of unique subbands = 94
Total number of subbands = 192
Total number of targets with ALMA data = 34
--------------------------------
